In [1]:
%reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, sqlite3, pickle, time, datetime


# 0 Loading Raw Data

In [2]:
t = time.time()
path  = r'/rfs/CAPAI_PhD_dflr2/Shared/Data/output/20220221_'
file1 = path + 'admissions.txt' 
file2 = path + 'eobs.txt'
file3 = path + 'haem-results.txt'
file4 = path + 'icu.txt'
file5 = path + 'meds.txt'
file6 = path + 'micro-results.txt'
file7 = path + 'oxygen.txt'
file8 = path + 'prev_admissions.txt'
file9 = path + 'spin.txt'
df_admin = pd.read_csv(file1, sep='\t', lineterminator='\n')
df_eobs  = pd.read_csv(file2, sep='\t', lineterminator='\n')
df_haemt = pd.read_csv(file3, sep='\t', lineterminator='\n')
df_icu   = pd.read_csv(file4, sep='\t', lineterminator='\n')
df_meds  = pd.read_csv(file5, sep='\t', lineterminator='\n')
df_micro = pd.read_csv(file6, sep='\t', lineterminator='\n')
df_oxyge = pd.read_csv(file7, sep='\t', lineterminator='\n')
df_prev  = pd.read_csv(file8, sep='\t', lineterminator='\n')
df_spin  = pd.read_csv(file9, sep='\t', lineterminator='\n')

/local/dlr10/ipykernel_7242/38138428.py:17: DtypeWarning: Columns (10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_micro = pd.read_csv(file6, sep='\t', lineterminator='\n')
/local/dlr10/ipykernel_7242/38138428.py:18: DtypeWarning: Columns (9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_oxyge = pd.read_csv(file7, sep='\t', lineterminator='\n')


In [3]:
display(df_admin.head(3))
print('min admin date', df_admin['admission_date'].min(),' max admin date', df_admin['admission_date'].max())
print('min disch date', df_admin['discharge_date'].min(),' max disch date', df_admin['discharge_date'].max())

,admission_id,episode_diagnoses,patient_id,admission_date,admission_time,discharge_date,discharge_time,death_indicator,date_of_death,ethnic_origin,sex,age_at_admission\r
0,49358678,"||I50.0,I10.X,J18.1,N18.9||I50.0,J18.1,I10.X,N...",74451425,2019-01-15 00:00:00,1909,2019-01-20 00:00:00,1500,YES,20210820.0,H,F,90
1,98363214,"||I50.0,J22.X,I51.7,I10.X,M19.9,M47.82,N19.X,E...",74451425,2019-04-23 00:00:00,417,2019-05-13 00:00:00,1800,YES,20210820.0,H,F,90
2,65069312,"||J18.1,J90.X,I50.0,I10.X,N17.9,R79.8,Z50.7,N1...",74451425,2020-01-04 00:00:00,558,2020-01-10 00:00:00,1820,YES,20210820.0,H,F,91


min admin date 2019-01-01 00:00:00  max admin date 2021-12-31 00:00:00
min disch date 2019-01-01 00:00:00  max disch date 2022-02-15 00:00:00


In [4]:
##############################################################################
#######################################
### Relevant fields for pre-processing
#######################################
reference_date = datetime.datetime(1970,1,1)

##### Fields in table patients ##########
patient_field         = 'patient_id'
age_field             = 'age_at_admission\r'
sex_field             = 'sex'
ethnic_field          = 'ethnic_origin'
death_ind_field       = 'death_indicator'
death_date_field      = 'date_of_death'
death_date_code_field = 'date_code_of_death'
mort_in_hosp_field    = 'Thirty_day_mort'
patients_cols = [patient_field,sex_field, ethnic_field, death_ind_field, death_date_field, death_date_code_field, 
                 mort_in_hosp_field]

##### Fields in table admissions ########
admission_field           = 'admission_id'
diagnosis_field           = 'episode_diagnoses'
admn_date_field           = 'admission_date_time'
admn_discharge_field      = 'discharge_date_time'
admn_date_code_field      = 'admission_date_code_time'
admn_discharge_code_field = 'discharge_date_code_time'
lengthofstay_field        = 'lengthofstay'
isPneumonia_field         = 'isPneumonia'
mortal_admin_field        = 'mortal_admin'
comorbidity_field           = 'Comorbidity_score'
icu_admin_field           = 'icu_count\r'

##### Fields in table eObservations #####
eObs_time_field      = 'timestamp'
eObs_time_code_field = 'timestamp_code'
eObs_resprate_field  = 'rr'
eObs_sbp_field       = 'sbp'
eObs_dbp_field       = 'dbp'
eObs_newscore_field  = 'ews'
eObs_heartrate_field = 'heart_rate'
eObs_temptr_field    = 'temperature\r'
eobs_cols = [eObs_time_field, eObs_time_code_field, eObs_resprate_field, eObs_sbp_field, eObs_dbp_field, eObs_newscore_field,
             eObs_heartrate_field, eObs_temptr_field]

##### Fields in table haematological tests #####
test_code_field      = 'local_test_code'
test_time_field      = 'sample_collection_date_time'
test_time_code_field = 'sample_collection_date_code_time'
haemt_cols = [test_code_field, test_time_field, test_time_code_field]

#######################################
#### ICD codes pre processing
#######################################
ICD_codes = 'A01.0, A02.2, A20.2, A21.2, A22.1, A31.0, A37.0, A37.1, A43.0, A48.1, B01.2, B05.2, B37.1, B38.0,' \
             'B38.1, B38.2, B39.0, B39.1, B39.2, B58.3, B59, B77.8, J12.0, J12.1, J12.2, J12.3, J12.8, J12.8,'\
             'J12.9, J13, J14, J15.0, J15.1, J15.2, J15.3, J15.4, J15.5, J15.6, J15.7, J15.8, J15.9,J16.0, J16.8, J17.0,'\
             'J17.1, J17.2, J17.3, J17.8, J18.0, J18.1, J18.8, J18.9, J85.1, J85.2, B59'
ICD_codes = re.sub(r"\s+", "", ICD_codes)
ICD_codes = ICD_codes.split(',')

# Charlson comorbidity codes source:
# https://cran.r-project.org/web/packages/comorbidity/vignettes/comorbidityscores.html

ICD_comorbidities = {}
# Myocardial Infarction
ICD_comorbidities[0] = ['I21.x', 'I22.x', 'I25.2']
# Congestive heart failure
ICD_comorbidities[1] = ['I09.9', 'I11.0', 'I13.0', 'I13.2', 'I25.5','I42.0','I42.5-  I42.9', 'I43.x', 'I50.x', 'P29.0']
# Peripheral vascular disease
ICD_comorbidities[2] = ['I70.x', 'I71.x', 'I73.1', 'I73.8', 'I73.9','I77.1', 'I79.0', 'I79.2', 'K55.1', 'K55.8', 'K55.9', 'Z95.8', 'Z95.9']
# Cerebrovascular disease
ICD_comorbidities[3] = ['G45.x', 'G46.x','H34.0', 'I60.x - I69.x']
# Dementia
ICD_comorbidities[4] = ['F00.x - F03.x', 'F05.1', 'G30.x', 'G31.1']
# Chronic Pulmonary Disease
ICD_comorbidities[5] = ['I27.8', 'I27.9','J40.x - J47.x', 'J60.x - J67.x', 'J68.4', 'J70.1', 'J70.3']
# Rheumatic Disease
ICD_comorbidities[6] = ['M05.x', 'M06.x', 'M31.5','M32.x - M34.x', 'M35.1', 'M35.3', 'M36.0']
# Peptic ulcer disease
ICD_comorbidities[7] = ['K25.x - K28.x']
# Mild liver disease
ICD_comorbidities[8] = ['B18.x', 'K70.0 - K70.3', 'K70.9','K71.3 - K71.5', 'K71.7', 'K73.x', 'K74.x', 'K76.0', 'K76.2 - K76.4', 'K76.8', 'K76.9','Z94.4']
# Diabetes without chronic complication
ICD_comorbidities[9] = ['E10.0', 'E10.1', 'E10.6', 'E10.8', 'E10.9', 'E11.0', 'E11.1', 'E11.6', 'E11.8','E11.9',' E12.0', 'E12.1', 'E12.6', 'E12.8', 'E12.9', 'E13.0', 'E13.1', 'E13.6', 'E13.8','E13.9', 'E14.0', 'E14.1', 'E14.6', 'E14.8', 'E14.9']
# Diabetes with chronic complication
ICD_comorbidities[10] = ['E10.2 - E10.5', 'E10.7', 'E11.2 - E11.5','E11.7', 'E12.2 - E12.5', 'E12.7', 'E13.2 - E13.5', 'E13.7', 'E14.2 - E14.5', 'E14.7']
# Hemiplegia or Paraplegia
ICD_comorbidities[11] = ['G04.1','G11.4', 'G80.1', 'G80.2', 'G81.x', 'G82.x', 'G83.0 - G83.4', 'G83.9']
# Renal Disease
ICD_comorbidities[12] = ['I12.0', 'I13.1','N03.2 - N03.7', 'N05.2 - N05.7', 'N18.x', 'N19.x', 'N25.0', 'Z49.0 - Z49.2', 'Z94.0', 'Z99.2']
# Any Malignancy except neoplasm of skin
ICD_comorbidities[13] = ['C00.x - C26.x', 'C30.x - C34.x', 'C37.x - C41.x', 'C43.x', 'C45.x - C58.x', 'C60.x - C76.x','C81.x - C85.x', 'C88.x', 'C90.x - C97.x']
# Moderate or severe liver disease
ICD_comorbidities[14] = ['I85.0', 'I85.9', 'I86.4', 'I98.2', 'K70.4', 'K71.1','K72.1', 'K72.9', 'K76.5', 'K76.6', 'K76.7']
# Metastatic solid tumor
ICD_comorbidities[15] = ['C77.x - C80.x']
# AIDS/HIV
ICD_comorbidities[16] = ['B20.x - B22.x', 'B24.x']
ICD_comorbidities_weights = dict(zip([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],
                                     [1,1,1,1,1,1,1,1,2,1,2 ,2 ,2 ,2 ,3 ,6 , 6]))

In [5]:
# Maximum score for charlson comoridities 
h =0
for i,v in ICD_comorbidities_weights.items(): h = h +v
h

34

# 1. Change fomart of categorical variables and dates


In [6]:
##############################################################################
# Processing admissions table times
df_admin['sex']                     = df_admin[sex_field].apply(lambda x: -1 if x == 'M' else 1)
df_admin['date_of_death']           = pd.to_datetime(df_admin['date_of_death'].apply(lambda x: str(int(x)) if not np.isnan(x) else '19900101'), format = '%Y%m%d')
df_admin['admission_date']          = pd.to_datetime(df_admin['admission_date'], format='%Y-%m-%d %H:%M:%S')
df_admin['admission_time']          = df_admin['admission_time'].apply(lambda x: datetime.timedelta(hours = int(x/100), minutes = x%100))
df_admin[admn_date_field]           = df_admin['admission_date'] + df_admin['admission_time']
df_admin[admn_date_code_field]      = df_admin[admn_date_field] - reference_date
df_admin[admn_date_code_field]      = df_admin[admn_date_code_field].apply(lambda x: x.days + (x.seconds/(24*3600)))
df_admin['discharge_date']          = pd.to_datetime(df_admin['discharge_date'], format='%Y-%m-%d %H:%M:%S')
df_admin['discharge_time']          = df_admin['discharge_time'].apply(lambda x: datetime.timedelta(hours = int(x/100), minutes = x%100))
df_admin[admn_discharge_field]      = df_admin['discharge_date'] + df_admin['discharge_time']
df_admin[admn_discharge_code_field] = df_admin[admn_discharge_field] - reference_date
df_admin[admn_discharge_code_field] = df_admin[admn_discharge_code_field].apply(lambda x: x.days + (x.seconds/(24*3600)))
df_admin[lengthofstay_field]        = df_admin[admn_discharge_field] - df_admin[admn_date_field]
df_admin[death_ind_field]           = df_admin[death_ind_field].apply(lambda x: 1 if x == 'YES' else 0 if x == 'NO' else 0)
df_admin = df_admin.drop(columns    = ['admission_date', 'admission_time','discharge_date','discharge_time'])

# Processing blood test table
df_haemt = df_haemt.rename(columns = {'system_number':  'patient_id' })
df_haemt[test_time_field]      = pd.to_datetime(df_haemt[test_time_field], format='%Y-%m-%d %H:%M:%S')
df_haemt[test_time_code_field] = df_haemt[test_time_field] - reference_date
df_haemt[test_time_code_field] = df_haemt[test_time_code_field].apply(lambda x: x.days + (x.seconds/(24*3600)))

# Marge all tables ina list call data
data = [df_admin, df_eobs, df_haemt, df_micro, df_prev]
print('time elapsed: ', time.time() - t)

time elapsed:  180.42700934410095


# 2. Create Patients Data Frame with demographics and patient specific information.

In [7]:
#######################################
#### Create Patient and Admisisons Table
#######################################
t = time.time()
pat_cols = [patient_field, sex_field, ethnic_field,death_ind_field, death_date_field, mort_in_hosp_field]

errors   = [] 
DF       = []
patients = df_admin[patient_field].unique().tolist()
pat      = patients[0]
for pat in patients:
    row = [pat]
    A   = df_admin[df_admin[patient_field] == pat].sort_values(by=admn_date_field)

    # ----------------------------
    # Sex of the patient
    if len(A[sex_field].unique()) == 1: row.append(A[sex_field].unique()[0]) 
    else: 
        errors.append(pat)
        continue
    # ----------------------------
    # Ethnic field of the patient
    if len(A[ethnic_field].unique()) == 1: row.append(A[ethnic_field].unique()[0]) 
    else: 
        errors.append(pat)
        continue
    # ----------------------------
    # Death indicator field of patient, whether the patient is died
    # Date of death field of patient
    if len(A[death_ind_field].unique()) == 1: 
        row.append(A[death_ind_field].unique()[0]) 
        row.append(A[death_date_field].unique()[0]) 
    else: 
        errors.append(pat)
        continue
    # ----------------------------
    # Thirty day mortality of apatient
    death = A.iloc[len(A)-1][death_date_field]
    disch = A.iloc[len(A)-1][admn_discharge_field]
    delt_day = datetime.timedelta(days=30)
    if death < disch + delt_day and row[-2] != 0 : row.append(1)
    else: row.append(0)
    DF.append(row)

df_patients = pd.DataFrame(DF, columns = pat_cols)
df_patients[death_date_code_field] = df_patients[death_date_field] - reference_date
df_patients[death_date_code_field] = df_patients[death_date_code_field].apply(lambda x: x.days + (x.seconds/(24*3600)))
df_patients   = pd.DataFrame(DF, columns = pat_cols)
print("Length of the table:", len(DF), ", Number of unique Patients:" ,len(patients), ", errors:",len(errors))
print("")
print('Female patients: ', len(df_patients[df_patients[sex_field] == 1]))
print('Male patients: '  , len(df_patients[df_patients[sex_field] == -1]))
print("")
print('patients mortality indicator 1:', df_patients[death_ind_field].sum())
print('patients mortality indicator 0:', len(df_patients) - df_patients[death_ind_field].sum())
print('Total of patients who died within 30 days:', df_patients[mort_in_hosp_field ].sum())
print("")
print('time elapsed: ', time.time() - t)

Length of the table: 20088 , Number of unique Patients: 20088 , errors: 0

Female patients:  9391
Male patients:  10697

patients mortality indicator 1: 8391
patients mortality indicator 0: 11697
Total of patients who died within 30 days: 5411

time elapsed:  136.34356904029846


# 3. Adding relevant fields to admissions table

In [8]:
t =  time.time()
df_admissions = df_admin.copy()
df_admissions = df_admissions.drop(columns =[ethnic_field,death_ind_field, death_date_field, sex_field])

#######################################
##### Definition of the functions
#######################################
#  that determines whether an admission is CAP associated or not
def isCAP(codes, ICD_codes):
    result = []
    for code in codes:
        classif = 0
        test = re.split(r'[,|\|\|]',code)
        A = set(ICD_codes) & set(test)
        if len(A) > 0: classif = 1
        result.append(classif)
    return result

# That determines whether the patient died in that admission or not according to the 30-day mortality indicator.
def admin_mortality(x, patients_df, admissions_df):
    pat = x[patient_field]
    if df_patients[df_patients[patient_field] == pat].iloc[-1][mort_in_hosp_field] == 0: return 0
    else:
        max_admin = admissions_df[admissions_df[patient_field] == pat].sort_values(by = admn_discharge_field).iloc[-1]
        if max_admin[admn_discharge_field] == x[admn_discharge_field]: return 1
        else: return 0

### 3.1. IsPneumonia and Mortal admission fields

In [9]:
#######################################
##### Creation of IsPneumonia and mortal admission fields
#######################################
codes = df_admissions[diagnosis_field].unique().tolist()
df_admissions[isPneumonia_field ] = isCAP(df_admissions[diagnosis_field], ICD_codes)
print('Number of admissions WITHOUT CAP ICD code:', len(df_admissions[df_admissions[isPneumonia_field] == 0]))
print('Number of admissions WITH    CAP ICD code:', len(df_admissions[df_admissions[isPneumonia_field] == 1]))
print("")
print('time elapsed: ', time.time() - t)

df_admissions[mortal_admin_field] = df_admissions.apply(lambda x: admin_mortality(x, df_patients, df_admissions), axis = 1)
#df_admissions[df_admissions[patient_field] == 62391666].sort_values(by=admn_discharge_field)
print('time elapsed: ', time.time() - t)

Number of admissions WITHOUT CAP ICD code: 0
Number of admissions WITH    CAP ICD code: 24720

time elapsed:  0.33472728729248047
time elapsed:  58.657936573028564


### 3. 2 ICU field

In [10]:
#######################################
##### Creation of ICU admission field
##### Wether the patient was admitted to ICU in given admission
#######################################
A = set(df_admissions[admission_field].unique().tolist())
print('Number of unique admissions in df_admissions ', len(A))
B = set(df_icu[admission_field].unique().tolist())
print('Number of unique admissions in df_icu ',len(B))
C = A & B
print('Number of admissions found in both df_admissions and df_icu',len(C))
icu_dict = dict(zip(df_icu[admission_field],df_icu[icu_admin_field]))
df_admissions[icu_admin_field] = df_admissions[admission_field].apply(lambda x: 0 if icu_dict[x] == 0 else 1)
print("People admited to ICU: " + str(df_admissions[icu_admin_field].sum()) )
print("People admited to ICU: " + str(df_admissions[icu_admin_field].sum()) )

Number of unique admissions in df_admissions  24720
Number of unique admissions in df_icu  24720
Number of admissions found in both df_admissions and df_icu 24720
People admited to ICU: 906
People admited to ICU: 906


### 3.3 Charlson comorbidity score

In [11]:
######################################################
# Comorbidity definition by Charlson Comorbidity. ICD_comorbidities list is the ICD10 codes that determine comorbidity
# The functions look at code of each admission determining whether it has a code 
t = time.time()
def is_code_in_group(code, ICD_comorbidities_group):
    comorbidity = False
    for com_code in ICD_comorbidities_group:
        if com_code == code:
            comorbidity = True
            break
        elif '-' in com_code:
            for sub_code in com_code.split('-'):
                sub_code = sub_code.strip()
                if sub_code == code:
                    comorbidity = True
                    break
                if sub_code[-1] == 'x':
                    sub_code = sub_code.split('.')[0]
                    if sub_code == code.split('.')[0]:
                        comorbidity = True
                        break
        elif com_code[-1] == 'x' or code[-1] == 'X':
            com_code = com_code.split('.')[0]
            if com_code == code.split('.')[0]:
                comorbidity = True
                break
        if comorbidity: break   
    return comorbidity      
def charlson_comorbidity_score(patient_codes, ICD_comorbidities):
    patient_codes = re.sub(r'(^\|\|)','',patient_codes)
    patient_codes = re.sub(r'(\|\|$)','',patient_codes)
    patient_codes = re.sub(r'(,$)','',patient_codes)
    patient_codes = re.sub(r'(\|\|)',',',patient_codes)
    patient_codes = re.sub(r'(,,)',',',patient_codes)
    patient_codes = re.split(',', patient_codes)
    score = 0
    comorb_codes_exclude = []
    for code in patient_codes:
        for idx, group_codes in ICD_comorbidities.items():
            if idx in comorb_codes_exclude: continue
            if is_code_in_group(code, group_codes):
                score = score + ICD_comorbidities_weights[idx]
                comorb_codes_exclude.append(idx)
                break
    return score   

df_admissions[comorbidity_field] = df_admissions[diagnosis_field].apply(lambda x: charlson_comorbidity_score(x, ICD_comorbidities))
print('time elapsed: ', time.time() - t)

time elapsed:  38.34792137145996


## DF_PATIENTS

In [12]:
print(len(df_patients[patient_field].unique()))
df_patients.head()

20088


,patient_id,sex,ethnic_origin,death_indicator,date_of_death,Thirty_day_mort
0,74451425,1,H,1,2021-08-20,0
1,18026138,1,A,0,1990-01-01,0
2,27754209,-1,A,0,1990-01-01,0
3,54047888,-1,H,1,2020-05-20,1
4,56373663,-1,S,0,1990-01-01,0


In [13]:
df_patients[df_patients[patient_field] == 51990214]

,patient_id,sex,ethnic_origin,death_indicator,date_of_death,Thirty_day_mort


## DF_ADMISSIONS

In [14]:
print(len(df_admissions[patient_field].unique()))
df_admissions.head()

20088


,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,mortal_admin,icu_count\r,Comorbidity_score
0,49358678,"||I50.0,I10.X,J18.1,N18.9||I50.0,J18.1,I10.X,N...",74451425,90,2019-01-15 19:09:00,17911.797917,2019-01-20 15:00:00,17916.625000,4 days 19:51:00,1,0,0,3
1,98363214,"||I50.0,J22.X,I51.7,I10.X,M19.9,M47.82,N19.X,E...",74451425,90,2019-04-23 04:17:00,18009.178472,2019-05-13 18:00:00,18029.750000,20 days 13:43:00,1,0,0,3
2,65069312,"||J18.1,J90.X,I50.0,I10.X,N17.9,R79.8,Z50.7,N1...",74451425,91,2020-01-04 05:58:00,18265.248611,2020-01-10 18:20:00,18271.763889,6 days 12:22:00,1,0,0,3
3,99645131,"||E10.5,L97.X,L08.9,R02.X,A41.9,R65.1,N17.9,E1...",18026138,55,2021-11-25 16:34:00,18956.690278,2021-12-31 14:48:00,18992.616667,35 days 22:14:00,1,0,0,2
4,21362780,"||U07.1,J12.8,B97.2,J15.9,B97.2,Z86.7||U07.1,J...",27754209,45,2021-11-12 17:21:00,18943.722917,2021-11-20 14:00:00,18951.583333,7 days 20:39:00,1,0,0,0


In [15]:

df_admissions.isna().sum()

admission_id                0
episode_diagnoses           0
patient_id                  0
age_at_admission\r          0
admission_date_time         0
admission_date_code_time    0
discharge_date_time         0
discharge_date_code_time    0
lengthofstay                0
isPneumonia                 0
mortal_admin                0
icu_count\r                 0
Comorbidity_score           0
dtype: int64

# 4. COVID review filter

In [16]:
###############################
# COVID FILTER
###############################
print('Number of admissions', len(df_admissions))
print('Number of patients', len(df_patients))

COVID_day_D  = pd.Timestamp(year=2020, month=2, day=1)
new_df_admin = df_admissions[df_admissions[admn_date_field] > COVID_day_D]
new_patients = new_df_admin[patient_field].unique()
new_df_patie = df_patients[df_patients[patient_field].isin(new_patients)]
print('Number of admissions no COVID', len(new_df_admin))
print('Number of patients no COVID', len(new_df_patie))

Number of admissions 24720
Number of patients 20088
Number of admissions no COVID 15001
Number of patients no COVID 13043


In [17]:
pickling_data = [new_df_patie, new_df_admin]
pickle.dump(pickling_data, open('DataFrame_pickles/df_patients_admissions_2019_2020.pickle', 'wb'))